## Constraint based analysis on small models

In this tutorial we will use `COBREXA`'s `flux_balance_analysis`, `flux_variability_analysis`, and `parsimonious_flux_balance_analysis` functions to analyze a toy model of *E. coli*.

Let's first load the model.

In [1]:
# download file if it is not already present
!isfile("e_coli_core.xml") && download("http://bigg.ucsd.edu/static/models/e_coli_core.xml", "e_coli_core.xml")

using COBREXA

model = load_model("e_coli_core.xml")

Metabolic model of type SBMLModel

⠀⠈⢀⠀⡀⠀⠀⠀⠀⡠⠂⠀⠀⠀⠀⠈⠀⠄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⠀⠀⠀⠀⢀⠐⡀⠀⠀⠀⠀⠄
⠀⠐⠀⠀⠀⠀⠀⠀⡠⠂⠀⠀⠀⠀⢰⠱⣀⠀⡄⢐⠀⠀⢀⠀⠀⠀⡂⠄⠔⠁⠰⠀⠠⠀⣆⠀⠄⢠⢀⠄
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠁⠠⠀⠀⠐⠀⠀⠀⠀⠀⠀⢀⠀⠀⠐⠀⠂⠀⠀⠀⠄⠀⠐⠀⢁⠄⠀⠀⠀⠀⠀
⠀⢀⠀⠐⡈⠀⡀⠀⠂⠀⣀⠀⠑⡈⢀⠀⠀⠀⠀⠀⡀⡠⠀⡀⠰⠁⠈⠂⠁⠀⠠⠀⠀⠂⡂⠀⠂⠂⠀⠀
⠠⠀⠐⠀⠂⠀⠀⢀⠀⠀⠀⠀⠊⠀⡐⠊⠐⠀⠀⠀⠀⠀⠐⠀⠂⠀⠀⠐⠀⠀⠀⠀⠀⠁⠃⠠⠀⠁⠐⠀
⠀⠠⠀⡀⠄⠀⠀⠂⠀⠀⠀⠠⠀⠠⠀⠀⠄⠀⠨⠀⠀⠀⠐⠀⠀⠄⢀⠀⠀⠀⠈⠀⠀⠀⠁⠄⠀⠀⠀⠀
⠀⢐⠐⠀⠄⠀⡂⠀⢐⠀⠀⠀⠀⠂⢀⢀⠐⠂⡀⠈⠀⠀⠀⠂⠀⠈⠀⡀⡐⠀⢄⠀⢀⠀⡆⠀⡀⣀⡀⡐
⠀⠈⠀⠀⠀⠀⠀⠐⢂⠀⢀⠀⠈⠀⠀⠀⠀⠀⠠⠀⠀⠠⠀⠀⠀⠈⠂⠀⠀⠀⠄⠐⠐⠀⠁⠀⠀⠑⠁⠀
⠂⠠⠀⠀⠀⠀⠀⠀⠀⢀⠀⠀⠠⠈⠀⠀⠀⠀⠀⠁⠀⠀⠠⠐⠀⠁⠈⠀⠁⢀⠀⠀⠀⠀⠀⠀⠀⠀⠌⠀
⠀⠀⠂⢨⠀⡀⠀⠐⠁⠐⠀⠐⠊⠀⠀⠀⠀⠀⠀⠀⠀⠀⠢⠒⠈⠐⠐⠁⠂⠀⠀⠀⠄⠓⠕⠂⠃⠁⠀⠐
⠠⠀⠨⠀⠁⠤⠄⠀⠁⡄⠀⠂⠠⠄⢈⠌⠠⠄⠀⢀⠀⠀⠀⠄⠨⠀⡤⠀⢀⠀⢀⠠⠀⠁⡔⠨⠀⠈⠄⠀
⠀⢀⢀⣀⠀⡠⡒⢀⢀⣀⠀⢀⣀⡀⢀⠀⢀⠀⡀⠀⡀⠀⠈⣀⠀⢀⣀⠀⡀⠀⢀⠁⢀⣀⣀⡀⡠⡀⡀⣀
⠀⠄⠀⠀⠀⠀⠀⠀⠀⠂⠁⠀⠀⠀⠀⠀⠀⣠⠀⠀⡀⠀⠀⠀⠀⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠐⠀⠀⠀⠀
⢀⠂⠀⠀⠂⠀⠈⠀⠐⠀⠀⠀⠁⠀⠀⠀⡀⠔⠑⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠢⠀⠀⡀⠂⠈⠀⠀⠀⠄
⠀⠐⠀⠀⡂⠀⠂⠀⠀⠀⠒⠐⠄⠂⠐⠀⠘⡀⠀⠠⡂⠃⠀⠂⠄⠂⠀⠀⠀⠀⡀⠀⡀⠀⡂⠂⠀⠀⢀⠀
Number of reactions: 95
Number of metabolites: 72


### Optimization solvers in `COBREXA`

To actually perform any optimization based analysis we need to load a solver. Any [`JuMP` supported solvers](https://jump.dev/JuMP.jl/stable/installation/#Supported-solvers) will work. Here we will use [`Tulip.jl`](https://github.com/ds4dm/Tulip.jl) to solve linear programs and [OSQP.jl](https://osqp.org/docs/get_started/julia.html) to solve quadratic programs.

In [4]:
using Tulip
using OSQP

lp_optimizer = Tulip.Optimizer
qp_optimizer = OSQP.Optimizer

OSQP.MathOptInterfaceOSQP.Optimizer

### Flux balance analysis

Most analysis functions come in two flavours that determine how the output will be returned. The suffix of the function determines this behaviour, e.g. `flux_balance_analysis_vec` returns a vector of fluxes in the order of the reactions returned by `reactions` (recall that this is one of the generic interface accessors), and `flux_balance_analysis_dict` that returns a dictionary mapping reaction ids to fluxes. 

In both cases there are two required inputs: the `model` and the `optimizer`.

In [5]:
vec_soln = flux_balance_analysis_vec(model, lp_optimizer)

95-element Vector{Float64}:
  -0.0
   6.007249566587288
   7.477381919012616
  -5.064375360924965
   0.2234617471308555
  -3.2148950303940156
   2.5043094327812505
  21.799492758453965
   4.959985078558465
   1.4969838027636577
   5.883903449309813e-8
   2.3921385258661974e-9
   4.959985078558436
   ⋮
   2.5510067318203676e-8
 -22.80983339309117
   6.007249566587296
   3.374152410464434e-7
  29.17582720267984
   9.04807316633683e-9
   4.819851209389017e-8
   9.952893763995706e-9
 -21.799492758453965
  -0.0
  -1.4330520742992535e-9
   3.2148950303940156

In [6]:
dict_soln = flux_balance_analysis_dict(model, lp_optimizer)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0
  "R_ACONTb"      => 6.00725
  "R_TPI"         => 7.47738
  "R_SUCOAS"      => -5.06438
  "R_GLNS"        => 0.223462
  "R_EX_pi_e"     => -3.2149
  "R_PPC"         => 2.50431
  "R_O2t"         => 21.7995
  "R_G6PDH2r"     => 4.95999
  "R_TALA"        => 1.49698
  "R_PPCK"        => 5.8839e-8
  "R_EX_lac__D_e" => 2.39214e-9
  "R_PGL"         => 4.95999
  "R_H2Ot"        => -29.1758
  "R_GLNabc"      => -0.0
  "R_EX_co2_e"    => 22.8098
  "R_EX_gln__L_e" => -0.0
  "R_EX_nh4_e"    => -4.76532
  "R_MALt2_2"     => -0.0
  "R_ME2"         => 1.49736e-7
  "R_GAPD"        => 16.0235
  "R_EX_akg_e"    => 3.85074e-9
  "R_CS"          => 6.00725
  "R_ETOHt2r"     => -1.43306e-9
  "R_ACKr"        => -3.41463e-9
  ⋮               => ⋮

### Problem modification

Often it is desirable to modify the problem before performing analysis. Problem modifications include things like changing the objective sense, the solver, the solver attributes, the flux constraints, and the optimization objective. For completeness we demonstrate all of the mentioned problem modifications below.

Note that these modifications are temporary and do not change the underlying `model`.

In [7]:
dict_soln = flux_balance_analysis_dict(
    model,
    qp_optimizer; # will change to Tulip below
    modifications = [ # modifications are evaluated in order
        change_objective("R_BIOMASS_Ecoli_core_w_GAM"),
        change_constraint("R_EX_glc__D_e", -12, -12),
        change_constraint("R_EX_o2_e", 0, 0),
        change_solver(lp_optimizer), # swap back to using Tulip
        change_solver_attribute("IPM_IterationsLimit", 110), # this is a Tulip specific attribute, other solvers have other attributes
        change_sense(COBREXA.MOI.MAX_SENSE), # COBREXA imports JuMP internally, other valid options here also include: COBREXA.MOI.MIN_SENSE
        ],
)

Dict{String, Float64} with 95 entries:
  "R_EX_fum_e"    => -0.0
  "R_ACONTb"      => 0.294088
  "R_TPI"         => 11.7289
  "R_SUCOAS"      => -5.31277e-11
  "R_GLNS"        => 0.069699
  "R_EX_pi_e"     => -1.00274
  "R_PPC"         => 0.781108
  "R_O2t"         => -2.65313e-17
  "R_G6PDH2r"     => 4.23154e-9
  "R_TALA"        => -0.0487648
  "R_PPCK"        => 5.66084e-11
  "R_EX_lac__D_e" => 1.86325e-10
  "R_PGL"         => 4.23153e-9
  "R_H2Ot"        => 8.2857
  "R_GLNabc"      => -0.0
  "R_EX_co2_e"    => -0.487021
  "R_EX_gln__L_e" => -0.0
  "R_EX_nh4_e"    => -1.48633
  "R_MALt2_2"     => -0.0
  "R_ME2"         => 1.40825e-10
  "R_GAPD"        => 23.2754
  "R_EX_akg_e"    => 6.82418e-11
  "R_CS"          => 0.294088
  "R_ETOHt2r"     => -9.78427
  "R_ACKr"        => -10.0729
  ⋮               => ⋮

### Flux variability analysis
Flux variability analysis can be performed serially or in parallel. Here the serial version is demonstrated, see the more advanced tutorials for the parallel implementation.

In [ ]:
fva_max, fva_min = flux_variability_analysis(model)

### Parsimonious flux balance analysis
Parsimonious flux balance analysis finds a unique flux solution that minimizes the sum of fluxes of the system. This function requires the use of a quadratic optimization solver (OSQP for this example). Like in `flux_balance_analysis`, two variants exist where the suffix determines the function output.

Note, if the solver used for the required optimizer input is not capable of solving quadratic programs then it is required to supply `qp_solver`. 

In [8]:
dict_soln = parsimonious_flux_balance_analysis_dict(
    model,
    lp_optimizer;
    modifications = [
        change_constraint("R_EX_glc__D_e", -12, -12),
        change_solver_attribute("IPM_IterationsLimit", 500), # modifies the required optimizer input
    ],
    qp_solver = change_solver(qp_optimizer), # only necessary if the first solver cannot handle QPs
    qp_solver_attributes = change_solver_attribute("verbose", false),
)

LoadError: MethodError: no method matching parsimonious_flux_balance_analysis_dict(::SBMLModel, ::Type{Tulip.Optimizer}; modifications=Function[COBREXA.var"#120#121"{String, Int64, Int64}("R_EX_glc__D_e", -12, -12), COBREXA.var"#116#117"{String, Int64}("IPM_IterationsLimit", 500)], qp_solver=COBREXA.var"#114#115"{DataType}(OSQP.MathOptInterfaceOSQP.Optimizer), qp_solver_attributes=COBREXA.var"#116#117"{String, Bool}("verbose", false))
[0mClosest candidates are:
[0m  parsimonious_flux_balance_analysis_dict([91m::StandardModel[39m, ::Any; modifications, qp_solver, qp_solver_attributes) at C:\Users\St. Elmo\.julia\packages\COBREXA\1LNff\src\analysis\parsimonious_flux_balance_analysis.jl:105

In [17]:
vec_soln = parsimonious_flux_balance_analysis_vec(
    model,
    lp_optimizer;
    modifications = [
        change_constraint("R_EX_glc__D_e", -12, -12),
        change_solver_attribute("IPM_IterationsLimit", 500), # modifies the required optimizer input
    ],
    qp_solver = change_solver(qp_optimizer), # only necessary if the first solver cannot handle QPs
    qp_solver_attributes = change_solver_attribute("verbose", false),
)

LoadError: MethodError: no method matching parsimonious_flux_balance_analysis_vec(::SBMLModel, ::Type{Tulip.Optimizer}; modifications=Function[COBREXA.var"#120#121"{String, Int64, Int64}("R_EX_glc__D_e", -12, -12), COBREXA.var"#116#117"{String, Int64}("IPM_IterationsLimit", 500)], qp_solver=COBREXA.var"#114#115"{DataType}(OSQP.MathOptInterfaceOSQP.Optimizer), qp_solver_attributes=COBREXA.var"#116#117"{String, Bool}("verbose", false))
[0mClosest candidates are:
[0m  parsimonious_flux_balance_analysis_vec([91m::StandardModel[39m, ::Any; modifications, qp_solver, qp_solver_attributes) at C:\Users\St. Elmo\.julia\packages\COBREXA\1LNff\src\analysis\parsimonious_flux_balance_analysis.jl:78